# Correction Partie 3

# Partie 3. Classification des textes : analyse de sentiments
Dans cette troisième partie du TP, nous allons développer quelques modèles pour l’analyse de sentiments (classement de textes). Le dataset utilisé est un dataset d’analyse de sentiments dans le contexte de la finance (https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis). Nous allons utiliser pandas, scikit-learn et nltk

In [ ]:
import nltk
import numpy  as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import re
# On tokenise nous même avec un espace blanc ou un caractère de ponctuation (\b)
token_pattern = re.compile(r'(?u)\b\w\w+\b')
tokenizer = token_pattern.findall

On télécharge le dataset (https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis?resource=download) et ajoute son path (ou autre méthode pour charger le fichier avec pandas)

## 1. Préparation de données
### Question
- Importer le dataset CSV fourni en utilisant pandas et afficher les 10 premiers échantillons.

Dans mon cas, j'utilise **Colab** je dépose donc mon fichier dans mon drive et renseigne le chemin pour y acceder. Dans votre cas si il tourne en local, renseignez juste le chemin jusqu’au fichier.

In [ ]:
# Si vous travaillez sur colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Le chemin jusqu’à "data.csv"
path_data = "/content/drive/MyDrive/PhD/Activities/Enseignements/Fabron DUT2 - Introduction IA/TP5-6/data.csv"

Maintenant on importe le csv avec pandas et affiche les 10 premiers échantillons avec la fonction *head()* de pandas.

In [ ]:
# On récupère le csv
data = pd.read_csv(path_data)
# On affiche les 10 premières lignes
data.head(10)

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral
5,$SPY wouldn't be surprised to see a green close,positive
6,Shell's $70 Billion BG Deal Meets Shareholder ...,negative
7,SSH COMMUNICATIONS SECURITY CORP STOCK EXCHANG...,negative
8,Kone 's net sales rose by some 14 % year-on-ye...,positive
9,The Stockmann department store will have a tot...,neutral


### Question
- Encoder les sentiments en utilisant **LabelBinarizer** pour avoir un encodage OneHot.

### Explications
LabelBinarizer est une classe du module scikit-learn en Python qui est utilisée pour convertir les étiquettes de classification multi-classes en une représentation binaire.

En d'autres termes, si vous avez des données étiquetées avec plusieurs classes, LabelBinarizer convertira chaque étiquette en un vecteur binaire correspondant. Chaque position du vecteur représentera une classe possible et sera soit 1 (si l'exemple appartient à cette classe) ou 0 (si l'exemple n'appartient pas à cette classe).

Par exemple, si vous avez un ensemble de données étiquetées avec les classes "*chat*", "*chien*" et "*oiseau*", LabelBinarizer convertira chaque étiquette en un vecteur binaire de longueur 3. Si un exemple est étiqueté comme "*chien*", le vecteur correspondant serait [0, 1, 0]. Si un exemple n'appartient à aucune des classes, le vecteur serait [0, 0, 0].

La sortie de LabelBinarizer peut être utilisée pour entraîner des modèles de classification binaires tels que les classificateurs de régression logistique ou de SVM, qui ne peuvent gérer que des étiquettes binaires (1 ou 0).

Dans notre cas nous souhaitons obtenir un résultat pour les labels "*positive*", "*negative*", "*neutral*" du style:

| Sentiment   | OneHot    |
|-------------|-----------|
| Positive    | [1, 0, 0] |
| Neutral     | [0, 1, 0] |
| Negative    | [0, 0, 1] |
| Autre token | [0, 0, 0] |

In [ ]:
# On crée une instance de LabelBinarizer
lb = LabelBinarizer()

# fit_transform() de LabelBinarizer pour encoder les sentiments
y_onehot = lb.fit_transform(data['Sentiment'])

On regarde ce qu'il se passe et compare avec nos sentiments en texte

In [ ]:
# On remarque que negative -> 0,0, 1; positive -> 1, 0, 0; neutral -> 0, 1, 0;
y_onehot, data['Sentiment']

(array([[0, 0, 1],
        [1, 0, 0],
        [0, 0, 1],
        ...,
        [0, 1, 0],
        [0, 1, 0],
        [0, 0, 1]]), 0       positive
 1       negative
 2       positive
 3        neutral
 4        neutral
           ...   
 5837    negative
 5838     neutral
 5839     neutral
 5840     neutral
 5841    positive
 Name: Sentiment, Length: 5842, dtype: object)

### Question
- Diviser le dataset en entrainement et test, en utilisant train_test_split, la taille du test est 30% et random_state=0.

### Aide
**train_test_split** est une fonction du module scikit-learn en Python qui permet de diviser un ensemble de données en un ensemble d'entraînement et un ensemble de test. Cette fonction est souvent utilisée dans le cadre de l'apprentissage automatique (machine learning) pour évaluer la performance d'un modèle sur des données indépendantes de celles utilisées pour l'entraînement.

La fonction train_test_split prend en entrée les données à diviser ainsi que le pourcentage de données à réserver pour l'ensemble de test. Elle renvoie quatre ensembles de données : *X_train, X_test, y_train, y_test*.

Les paramètres les plus couramment utilisés de train_test_split sont :
- **test_size** : le pourcentage de données à réserver pour l'ensemble de test. Par exemple, si test_size est fixé à 0.2, cela signifie que 20% des données seront réservées pour le test et que 80% des données seront utilisées pour l'entraînement.
- **random_state** : un entier qui permet de fixer la graine aléatoire pour la répartition des données. Cela permet de s'assurer que la même répartition des données sera obtenue à chaque exécution du code, ce qui permet de faciliter la reproductibilité des résultats.

### Rappel:
Dans le contexte de l'apprentissage automatique supervisé, X représente les variables explicatives (ou caractéristiques) du jeu de données et y représente la variable cible (ou réponse) que nous cherchons à prédire.

In [ ]:
# Stocker les labels "Sentence" dans la variable X
X = data["Sentence"]

# Diviser l'ensemble de données en ensembles d'entraînement et de test en utilisant la fonction train_test_split de scikit-learn. 
# L'ensemble de test contiendra 30% des données et la graine aléatoire est fixée à 0 pour assurer la reproductibilité des résultats.
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.3, random_state=0)

Si vous êtes curieux ou dubitatif, vérifiez les données dans X et y

In [ ]:
# print(X_train, X_test, y_train, y_test)

Puis verifiez le ratio 70% train 30% test ainsi que la correspondance de taille X-y

In [ ]:
# On check les tailles si on a bien 70/30% 
print(len(X_train))
print(y_train.shape[0])
print(len(X_test))
print(y_test.shape[0])

4089
4089
1753
1753


### Question 
- Entrainer un modèle de vectorisation TF sur le texte d’entrainement et vectoriser le.

### Aide
On utilisera **CountVectorizer()**, qui est une fonction de la bibliothèque Scikit-Learn utilisée pour transformer un corpus de documents textuels en une matrice de termes de document, qui représente la fréquence des mots dans chaque document.

Plus précisément, CountVectorizer() :

- Tokenise les documents : chaque document est divisé en mots ou "tokens".
- Construit un vocabulaire : les mots uniques de tous les documents sont collectés pour créer un vocabulaire.
- Encode chaque document : chaque document est représenté par un vecteur qui compte la fréquence de chaque mot du vocabulaire dans ce document.
Le résultat est une matrice creuse (sparse matrix) où chaque ligne correspond à un document et chaque colonne correspond à un mot du vocabulaire. Les valeurs dans la matrice représentent le nombre d'occurrences de chaque mot dans chaque document.

Voici un exemple avec les 3 documents suivants:
- Le chien rouge joue avec le chat rouge
- Le chien est bleu, le chat est rouge, le chien est rouge
- Le chat est rouge

| Le         | chien | rouge | joue | avec | le | chat | est | bleu |
|------------|-------|-------|------|------|----|------|-----|------|
| Document 1 | 1     | 1     | 2    | 1    | 1  | 1    | 1   | 0    |
| Document 2 | 1     | 2     | 2    | 0    | 0  | 1    | 1   | 2    |
| Document 3 | 1     | 0     | 1    | 0    | 0  | 0    | 1   | 1    |

Cette matrice peut ensuite être utilisée comme entrée pour des algorithmes de machine learning tels que la régression logistique, les SVM, etc.

In [ ]:
# Création d'une instance de CountVectorizer
vectorizer = CountVectorizer()

# Utilisation de fit_transform() pour vectoriser le texte d'entraînement
vectorise = vectorizer.fit_transform(X_train)

Regardons un exemple de plus pres avec le premier document: "The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model ."

In [ ]:
# On prends un exemple
exemple = X_train.loc[0]
# On affiche la sparse matrice de notre phrase de test
print("La sparse matrice du document:", exemple)
print(vectorizer.transform([exemple]))

La sparse matrice du document: The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model .
  (0, 1013)	2
  (0, 1343)	1
  (0, 1417)	1
  (0, 1723)	1
  (0, 2107)	1
  (0, 2127)	1
  (0, 2280)	1
  (0, 3907)	1
  (0, 3989)	1
  (0, 5122)	1
  (0, 5231)	2
  (0, 5661)	1
  (0, 5754)	1
  (0, 5864)	1
  (0, 6505)	1
  (0, 6613)	1
  (0, 6793)	1
  (0, 7102)	1
  (0, 7567)	1
  (0, 7931)	1
  (0, 8449)	2
  (0, 8539)	1
  (0, 9297)	1


Par exemple, le tuple (0, 1013) 2 signifie que le mot ayant l'indice 1013 dans le vocabulaire du texte d'entraînement (c'est-à-dire le mot correspondant à la 1014ème colonne de la matrice) apparaît deux fois dans la première phrase du texte d'entraînement.

Pour afficher les indices de chaque mot dans le vocabulaire correspondant aux colonnes de la matrice, vous pouvez utiliser l'attribut vocabulary_ de l'objet vectorizer, comme ceci :

In [ ]:
# Afficher le dictionnaire
X_voc = vectorizer.vocabulary_
print(X_voc)

# Si vous chercher une valeur en particulier dans le vocabulaire
valeur_recherchee = 1013
for cle, valeur in X_voc.items():
    if valeur == valeur_recherchee:
        print("L'index:", valeur_recherchee, "correspond au mot:", cle)

{'maximum': 5485, 'of': 6037, '666': 553, '104': 44, 'new': 5864, 'shares': 7697, 'can': 1771, 'further': 3822, 'be': 1368, 'subscribed': 8210, 'for': 3703, 'by': 1723, 'exercising': 3376, 'options': 6126, 'under': 8886, 'the': 8539, '2004': 205, 'stock': 8122, 'option': 6124, 'plan': 6494, 'sellers': 7616, 'were': 9252, 'founders': 3748, 'company': 2144, 'we': 9217, 'are': 1108, 'pleased': 6514, 'to': 8643, 'welcome': 9242, 'tapeks': 8408, 'noma': 5925, 'into': 4598, 'cramo': 2392, 'group': 4025, 'board': 1521, 'directors': 2701, 'would': 9363, 'propose': 6766, 'dividend': 2760, 'eur0': 3181, '15': 103, 'per': 6387, 'share': 7690, '2005': 206, 'sales': 7442, 'tiimari': 8608, 'segment': 7600, 'fell': 3542, 'year': 9405, 'on': 6079, 'eur3': 3241, 'in': 4424, 'june': 4788, '2010': 217, 'yit': 9423, 'corporation': 2341, 'sept': 7644, '24': 258, '2007': 209, 'at': 1185, '13': 78, '30': 326, 'corporate': 2340, 'release': 7099, 'exchange': 3362, 'capital': 1791, 'markets': 5446, 'day': 2512,

- En utilisant le même modèle, vectoriser le dataset de test.

### Aide
La méthode fit_transform() de CountVectorizer est une combinaison des méthodes fit() et transform(). Cette méthode est utilisée pour apprendre le vocabulaire du texte d'entraînement et vectoriser le texte en une matrice creuse (sparse matrice). Elle est utilisée pour la première étape de la création d'un modèle de classification de texte, où le vocabulaire est appris à partir du texte d'entraînement et utilisé pour transformer le texte d'entraînement en une représentation vectorielle.

La méthode transform() de CountVectorizer, quant à elle, est utilisée pour transformer le texte de test (ou tout autre texte) en une représentation vectorielle en utilisant le vocabulaire appris pendant la phase d'entraînement. Cette méthode est utilisée pour vectoriser le texte de test de la même manière que le texte d'entraînement, en utilisant le même vocabulaire.

**En résumé**, fit_transform() est utilisée pour apprendre le vocabulaire et vectoriser le texte d'entraînement, tandis que transform() est utilisée pour vectoriser le texte de test en utilisant le même vocabulaire que celui appris pendant la phase d'entraînement. La méthode fit_transform() doit être utilisée sur le texte d'entraînement, tandis que la méthode transform() doit être utilisée sur le texte de test (ou tout autre texte à vectoriser) après avoir utilisé fit_transform() sur le texte d'entraînement.

In [ ]:
# Utilisation de transform() pour vectoriser le texte de test
vectorise_test = vectorizer.transform(X_test)

### Question
- Définir une fonction tokenstem qui prend un texte et qui utilise tokenizer pour avoir une liste des tokens ensuite elle génère une liste des tokens radicalisés en utilisant nltk.stem.porter.PorterStemmer et elle ne considère pas les tokens appartenant à nltk.corpus.stopwords.words(’english’)


On télécharge les données, les stop words en anglais cette fois ci:

In [ ]:
%%capture
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Maintenant on crée notre fonction **tokenstem** (tokenisation + remove stopwords + stemming)

In [ ]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

def tokenstem(text):
    # Tokenisation des mots dans le texte
    tokens = tokenizer(text) # On utilise notre fonction de tokenisation défini plus haut
    # Création de l'objet PorterStemmer
    # Il y a d'autres méthodes que PorterStemmer, vous pouvez comparer
    stemmer = PorterStemmer()
    # Liste de stopwords en anglais
    stop_words = set(stopwords.words('english'))
    # Racinisation des mots et exclusion des stopwords
    stems = [stemmer.stem(word) for word in tokens if not word.lower() in stop_words]
    return stems

On regarde le résultat de note superbe fonction **tokenstem**

In [ ]:
print(X_train.loc[0])
print(tokenstem(X_train.loc[0]))

The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model .
['geosolut', 'technolog', 'leverag', 'benefon', 'gp', 'solut', 'provid', 'locat', 'base', 'search', 'technolog', 'commun', 'platform', 'locat', 'relev', 'multimedia', 'content', 'new', 'power', 'commerci', 'model']


### Question
-  Refaire les deux étapes précédentes, mais en limitant la taille du vecteur max_features = 3000 et en utilisant tokenstem comme tokenizer.

On utilise toujours **CountVectorizer()**, regardez comment l'adapter avec nos rpérequis.

In [ ]:
# Création d'une instance de CountVectorizer avec un maximum de 3000 features et en utilisant le tokenizer personnalisé (question précédente)
vectorizer = CountVectorizer(max_features=3000, tokenizer=tokenstem)

# Utilisation de fit_transform() pour vectoriser le texte d'entraînement
vectorise_custom = vectorizer.fit_transform(X_train)
# Utilisation de transform() pour vectoriser le texte de test
vectorise_test_custom = vectorizer.transform(X_test)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## 2. Entrainement
### Question
- Créer un modèle de type linear support vector machine (SVM)https://scikit-learn.org/stable/modules/svm.html#svm
- Prédire les classes en appliquant le modèle sur les représentations d’entrainement
- Prédire les classes en appliquant le modèle sur les représentations vectorielles de test

### Rappel
**SVM**, ou Support Vector Machines, est un algorithme d'apprentissage automatique supervisé utilisé pour la classification et la régression. Il est principalement utilisé pour résoudre des problèmes de classification binaire, où l'objectif est de séparer les données en deux classes distinctes.

Le principe de base de SVM consiste à trouver un hyperplan qui sépare les données en deux classes. L'hyperplan est choisi de manière à maximiser la marge entre les deux classes, c'est-à-dire la distance entre les points les plus proches des deux classes. Les points les plus proches des deux classes sont appelés vecteurs de support, d'où le nom de "Support Vector Machines".

Le type "*linear*" de SVM est utilisé lorsque les données peuvent être séparées linéairement, c'est-à-dire lorsque les deux classes peuvent être séparées par un hyperplan. Dans ce cas, l'algorithme de SVM trouve l'hyperplan optimal qui maximise la marge entre les deux classes. L'hyperplan est défini par une fonction linéaire qui prend les caractéristiques d'entrée des données en entrée et produit une sortie qui permet de déterminer la classe à laquelle appartient chaque donnée.

On implemente un SVM avec la première méthode d'entrainement des embeddings.
Vous aurez surement besoin d'adapter les données avec la fonction inverse_transform() pour recuperer les labels initiaux.

In [ ]:
from sklearn.svm import SVC

# Définir le modèle SVM
svm = SVC(kernel='linear')

y_train_ = lb.inverse_transform(y_train)
# Entraîner le modèle
svm.fit(vectorise, y_train_)

# Prédire les classes pour les données de test
y_pred = svm.predict(vectorise_test)

- Afficher le rapport de classification

On utilisera **classification_report**, une fonction de la bibliothèque scikit-learn en Python qui calcule les métriques de performance pour un modèle de classification. Elle prend en entrée les vraies étiquettes de classe (appelées également labels ou ground truth) et les prédictions du modèle, puis calcule un certain nombre de métriques de performance telles que la précision, le rappel, le score F1 et le support.

Les métriques de performance calculées par classification_report sont spécifiques à chaque classe. Pour chaque classe, elle calcule les métriques de performance suivantes:

- La précision (precision) : la proportion de prédictions positives correctes par rapport à toutes les prédictions positives effectuées par le modèle. Elle mesure la capacité du modèle à identifier correctement les instances de la classe donnée.

- Le rappel (recall) : la proportion d'instances positives correctement identifiées par rapport à toutes les instances positives dans les données. Elle mesure la capacité du modèle à identifier toutes les instances positives de la classe donnée.

- Le score F1 (F1 score) : une moyenne harmonique de la précision et du rappel qui donne un score unique pour chaque classe. Il est utile lorsque la précision et le rappel ont des valeurs très différentes.

- Le support (support) : le nombre d'instances réelles de la classe donnée dans les données.

classification_report calcule également ces métriques pour l'ensemble des classes, en prenant en compte la moyenne des métriques sur toutes les classes.

In [ ]:
# Évaluer la performance du modèle
print(classification_report(lb.inverse_transform(y_test), y_pred))

              precision    recall  f1-score   support

    negative       0.25      0.21      0.23       283
     neutral       0.70      0.75      0.72       940
    positive       0.72      0.69      0.71       530

    accuracy                           0.64      1753
   macro avg       0.56      0.55      0.55      1753
weighted avg       0.63      0.64      0.64      1753



### Question
- Créer un modèle de type linear support vector machine (SVM)https://scikit-learn.org/stable/modules/svm.html#svm
- Prédire les classes en appliquant le modèle sur les représentations d’entrainement
- Prédire les classes en appliquant le modèle sur les représentations vectorielles de test

Cette fois ci on experimente avec notre foction tokenstem

In [ ]:
# Définir le modèle SVM
svm_custom = SVC(kernel='linear')

y_train_ = lb.inverse_transform(y_train)
# Entraîner le modèle
svm_custom.fit(vectorise_custom, y_train_)

# Prédire les classes pour les données de test
y_pred_custom = svm_custom.predict(vectorise_test_custom)

In [ ]:
# Évaluer la performance du modèle
print(classification_report(lb.inverse_transform(y_test), y_pred_custom))

              precision    recall  f1-score   support

    negative       0.24      0.22      0.23       283
     neutral       0.68      0.72      0.70       940
    positive       0.67      0.63      0.65       530

    accuracy                           0.61      1753
   macro avg       0.53      0.52      0.53      1753
weighted avg       0.61      0.61      0.61      1753



# Fin
Libre a vous de continuer et d'experimenter avec d'autresfonctions et paramètres. Vous pouvez par exemple utiliser d'autres méthodes de vectorisation comme: 
TfidfVectorizer : Cette fonctionnalité est également disponible dans la bibliothèque scikit-learn. Elle utilise le schéma de pondération TF-IDF pour la vectorisation de texte, qui est une technique courante pour représenter les termes dans un document en fonction de leur importance. Elle attribue des poids plus élevés aux termes qui sont fréquents dans un document et rares dans l'ensemble du corpus.

- HashingVectorizer : Cette fonctionnalité est également disponible dans la bibliothèque scikit-learn. Elle utilise une fonction de hachage pour convertir les mots en vecteurs de nombres entiers de longueur fixe. Cette fonctionnalité est plus rapide et nécessite moins de mémoire que CountVectorizer car elle ne stocke pas de vocabulaire.

- Word2Vec : Cette fonctionnalité utilise un modèle de réseau de neurones pour apprendre des représentations vectorielles denses pour les mots. Elle est capable de capturer des similitudes sémantiques et syntaxiques entre les mots et est utile pour la résolution de tâches telles que la prédiction de mots manquants et la classification de textes.

- GloVe : Cette fonctionnalité utilise une approche matricielle pour apprendre des représentations vectorielles pour les mots. Elle est similaire à Word2Vec et est également utile pour la résolution de tâches telles que la classification de textes et la prédiction de mots manquants.

- FastText : Cette fonctionnalité est développée par Facebook et utilise une méthode de n-grammes pour apprendre des représentations vectorielles pour les mots. Elle est capable de capturer des similitudes entre les mots qui partagent des sous-chaînes et est utile pour la classification de textes et la prédiction de mots manquants.

# Partie 4. Classification des textes : analyse de sentiments (suite)
Dans ce TP, nous allons continuer à explorer quelques modèles pour l’analyse de sentiments (classement de textes). On utilisera toujours le même dataset d’analyse de sentiments dans le contexte de la finance (https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis), et les mêmes librairies de la semaine dernière (pandas, scikit- learn et nltk).

In [ ]:
# On renome juste les variables pour s'y retrouver
X_train_vectorized = vectorise
X_test_vectorized = vectorise_test

## 1. Entrainement
### Question
- Créer un modèle de type MLPClassifier avec une couche cachée et un maximum d’itérations de 50.

**MLPClassifier** est une classe d'algorithmes de classification basée sur les réseaux de neurones artificiels, qui sont également connus sous le nom de perceptrons multicouches. Cette classe est implémentée dans la bibliothèque de machine learning scikit-learn en Python.

Une *couche cachée* dans un MLPClassifier est une couche de neurones qui se trouve entre la couche d'entrée et la couche de sortie. Les neurones de la couche cachée effectuent une transformation non linéaire de la sortie de la couche précédente avant de la passer à la couche suivante. Les couches cachées sont nécessaires pour permettre au modèle de capturer des relations non linéaires entre les variables d'entrée et la variable cible.

Le nombre de couches cachées et le nombre de neurones dans chaque couche sont des *hyperparamètres* qui doivent être déterminés par essais et erreurs pour obtenir la meilleure performance du modèle. Dans le cas de MLPClassifier, le nombre de couches cachées est déterminé par le paramètre *hidden_layer_sizes*, qui prend une liste de nombres entiers, chacun représentant le nombre de neurones dans chaque couche cachée.

Le paramètre *max_iter* dans MLPClassifier spécifie le nombre maximum d'itérations que l'algorithme doit effectuer avant de s'arrêter. Chaque itération correspond à une mise à jour des poids du modèle basée sur les données d'entraînement. Si le modèle ne converge pas avant que le nombre maximum d'itérations soit atteint, l'algorithme s'arrête et renvoie le modèle actuel. Un nombre plus élevé d'itérations peut conduire à un meilleur ajustement des données d'entraînement, mais peut également augmenter le risque de surajustement.

In [1]:
# Création du modèle MLPClassifier
n = 100  # Nombre de neurones dans la couche cachée unique, 100 par exemple
# TODO

### Question 
- Entrainer le modèle sur les représentations vectorielles obtenues comme résultat de l’étape 1 de la semaine dernière.

Utilisez la variable que nous avons préalablement renommée.

In [2]:
# Entraînement du modèle
# TODO

### Question 
- Prédire les classes en appliquant le modèle sur les représentations d’entrainement.

In [3]:
# Prédiction du modèle sur les données d'entrainement
# TODO

### Question
- Afficher le rapport de classification.

Vous pouvez utiliser le paramètre `target_names=["negative", "neutral", "positive"]` pour nommer les prédictions.

In [4]:
# Rapport de classification pour les données d'entraînement
# TODO

# TODO print

Que remarquez vous ici ? Rien d'étrange ?

In [ ]:
# Avoir un résultat de 1.00 ou 100% est quasiment impossible et est souvent significatif d'un sur-apprentissage du modèle.

## 2. Test
### Question 
- Prédire les classes en appliquant le modèle sur les représentations vectorielles de test.

In [5]:
# Prédiction du modèle sur les données de test
# TODO

### Question
- En utilisant timeit, calculer le temps de prédiction.

In [7]:
import timeit

# Définition d'une fonction pour encapsuler la prédiction
def predict_mlp():
  return
    # TODO like mlp_classifier.predict(X_test_vectorized)

# Calcul du temps de prédiction en utilisant timeit.timeit()
num_runs = 1000  # Nombre d'exécutions pour obtenir une estimation plus ou moins précise du temps
# TODO

# Calcul du temps moyen de prédiction par exécution
# TODO

# TODO print

### Question
- Afficher le rapport de classification.

In [8]:
# Rapport de classification
# TODO

# TODO print

### Question
- Comparer les résultats obtenus avec ce modèle aux resultats obtenus avec SVM : qu’observez-vous ?
- Comparez les résultats de test avec les résultats de train, qu'en pensez vous?

Dans votre cas, il y a une différence importante entre les scores d'entraînement et de test. Cela suggère que le modèle pourrait souffrir de surapprentissage, ce qui signifie qu'il a appris les données d'entraînement "par cœur" et ne parvient pas à généraliser correctement sur de nouvelles données. Pour améliorer les performances du modèle, vous pouvez essayer les approches suivantes :

- Utiliser une régularisation (par exemple, L1 ou L2) pour pénaliser les poids du modèle et réduire la complexité du modèle.
- Augmenter la taille de l'ensemble de données pour donner au modèle plus d'exemples à apprendre.
- Réduire la complexité du modèle en ajustant le nombre de neurones ou de couches cachées.
- Utiliser d'autres techniques de prétraitement des données, comme le TF-IDF, pour représenter les données textuelles.
- Optimiser les hyperparamètres du modèle en utilisant la validation croisée et une recherche sur grille.

## Partie 5. Améliorations possibles
Nous voulons maintenant améliorer ce modèle, qui a quelques inconvénients (rien n’est parfait). Dans cette partie du TP, vous devez étudier quelques limites de ce modèle et les améliorer. Il faut au moins améliorer une chose sans détériorer le reste (une petite détérioration est acceptable).
- taille d’encodage : un modèle avec moins de paramètres est préférable
- Précision (micro-avg)
- Rappel (micro-avg)
- Temps de test : utile lorsqu’on veut utiliser ce modèle avec une application qui doit être rapide comme moteur de recherche.

### Question
- Vous pouvez changer d’algorithme d’apprentissage : utiliser les autres algorithmes de scikit-learn
- Vous pouvez paramétrer le MLP existant : ajouter des couches, changer la fonction d’activation, etc.
- Vous pouvez utiliser d’autres méthodes de vectorisation (quelques modèles entrainés sont fournis avec ce TP)

In [9]:
# Création du modèle MLPClassifier avec d'autres paramètres 
# TODO

Changer les paramètres *hidden_layer_sizes* et *activation* dans un réseau de neurones tel que MLPClassifier peut avoir un impact significatif sur la performance et la complexité du modèle.

- hidden_layer_sizes: C'est le nombre de neurones dans chaque couche cachée du réseau. Augmenter le nombre de neurones peut permettre au modèle de capturer des motifs plus complexes dans les données, mais cela peut également augmenter le risque de surapprentissage si le nombre de neurones est trop élevé. En général, il est recommandé de commencer avec une petite quantité de neurones et de les augmenter progressivement si nécessaire pour améliorer la performance du modèle.

- activation: C'est la fonction d'activation utilisée par les neurones dans le réseau. Elle détermine la manière dont les valeurs d'entrée sont transformées en sorties. Par exemple, la fonction d'activation "relu" (Rectified Linear Unit) permet d'obtenir une sortie égale à zéro pour toute valeur d'entrée négative et une sortie égale à l'entrée pour toute valeur d'entrée positive. La fonction d'activation "tanh" (tangente hyperbolique) a des valeurs de sortie dans l'intervalle [-1,1] et est utile pour les problèmes de classification binaire. La liste des fonctions d'activation disponible dans MLPClassifier est la suivante: 'tanh', 'relu', 'identity', 'logistic'

In [10]:
# # Création du modèle MLPClassifier avec d'autres paramètres 
# n = 20  # Nombre de neurones dans les couches cachées
# # Vouc pouvez changer activation parmis 'tanh', 'relu', 'identity', 'logistic'
# mlp_classifier_updates = MLPClassifier(hidden_layer_sizes=(n, n, n,), activation=('relu'), max_iter=50, random_state=0)

# Entraînement du modèle
# TODO

# Prédiction
# TODO

# Rapport de classification
# TODO

# TODO print

Implementons l'algorithme kNN (k-Nearest Neighbors). kNN est une méthode d'apprentissage supervisé pour la classification et la régression. L'approche kNN est basée sur l'idée que les exemples similaires tendent à se regrouper dans des régions similaires de l'espace de caractéristiques.

Dans le cas de la classification, l'algorithme kNN fonctionne de la manière suivante :

Tout d'abord, l'algorithme est entraîné sur un ensemble de données étiqueté, qui consiste en un ensemble d'exemples d'entrée avec leurs étiquettes de classe correspondantes.

Lors de la prédiction d'une nouvelle instance, l'algorithme kNN calcule la distance entre la nouvelle instance et tous les exemples d'entraînement.

L'algorithme sélectionne ensuite les k exemples d'entraînement les plus proches de la nouvelle instance en termes de distance.

Enfin, l'algorithme assigne une étiquette de classe à la nouvelle instance en fonction de la majorité des étiquettes de classe parmi les k exemples les plus proches.

La valeur de k est un paramètre important de l'algorithme kNN, car elle détermine le nombre d'exemples d'entraînement qui seront considérés pour la prédiction de la nouvelle instance. Une valeur de k plus élevée réduira la sensibilité aux valeurs aberrantes mais peut également causer une erreur de classification pour les classes frontalières.

L'algorithme kNN peut également être utilisé pour la régression, où la valeur de sortie est une variable numérique continue plutôt qu'une variable de classe discrète. Dans ce cas, l'algorithme sélectionne les k exemples les plus proches de la nouvelle instance et utilise leur valeur de sortie moyenne pour prédire la valeur de sortie de la nouvelle instance.

L'algorithme kNN est simple à mettre en œuvre et peut être efficace dans de nombreux cas d'utilisation, mais il peut également être sensible aux valeurs aberrantes et nécessite souvent une normalisation des données avant l'entraînement.

In [11]:
from sklearn.neighbors import KNeighborsClassifier

# Création de l'instance kNN
k = 3
# TODO

# TODO print

TfidfVectorizer est une classe de la bibliothèque scikit-learn qui permet de convertir une collection de documents texte en une matrice de termes de document. TfidfVectorizer effectue à la fois la tokenisation (séparation des mots) et le calcul du poids de chaque terme dans chaque document en utilisant le schéma de pondération tf-idf.

Le schéma de pondération tf-idf (term frequency-inverse document frequency) est une mesure de l'importance d'un terme dans un document par rapport à sa fréquence dans l'ensemble du corpus. La fréquence du terme (tf) est simplement le nombre d'occurrences du terme dans le document, tandis que l'inverse de la fréquence du document (idf) est une mesure de la rareté du terme dans l'ensemble du corpus.

La formule pour le calcul du poids tf-idf d'un terme dans un document est la suivante :

```
tf-idf = tf * log(N / df)
```


où N est le nombre total de documents dans le corpus et df est le nombre de documents dans le corpus qui contiennent le terme.

TfidfVectorizer effectue cette opération pour chaque terme dans chaque document et renvoie une matrice de termes de document, où chaque ligne représente un document et chaque colonne représente un terme, et la valeur dans chaque case est le poids tf-idf du terme dans le document correspondant.

TfidfVectorizer offre plusieurs options pour personnaliser le traitement des documents et des termes, telles que la suppression des stopwords (mots courants qui ne sont pas considérés comme des termes importants), la normalisation des vecteurs de terme-document et la gestion des n-grammes (séquences contiguës de mots) dans les documents.

TfidfVectorizer est couramment utilisé pour la classification de texte et le regroupement de documents, car il permet de représenter les documents de manière numérique tout en conservant les informations sémantiques importantes.


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Création d'une instance de TfidfVectorizer
# TODO

In [ ]:
# Création du modèle MLPClassifier avec d'autres paramètres 
n = 100  # Nombre de neurones dans les couches cachées
# Vouc pouvez changer activation parmis 'tanh', 'relu', 'identity', 'logistic'
mlp_classifier_updates = MLPClassifier(hidden_layer_sizes=(n, n, n,), activation=('relu'), max_iter=50, random_state=0)

# Entraînement du modèle
mlp_classifier_updates.fit(X_train_tfidf_vectorise, y_train)

# Prédiction
y_pred = mlp_classifier_updates.predict(X_test_tfidf_vectorise_test)

# Rapport de classification
report = classification_report(y_test, y_pred, target_names=["negative", "neutral", "positive"])

print("Rapport de classification :")
print(report)

Rapport de classification :
              precision    recall  f1-score   support

    negative       0.26      0.18      0.21       283
     neutral       0.68      0.75      0.71       940
    positive       0.69      0.66      0.68       530

   micro avg       0.63      0.63      0.63      1753
   macro avg       0.54      0.53      0.53      1753
weighted avg       0.61      0.63      0.62      1753
 samples avg       0.62      0.63      0.63      1753



/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### A vous de jouer !
Scikit-learn est une bibliothèque open-source pour l'apprentissage automatique en Python. Elle propose une large gamme d'algorithmes d'apprentissage supervisé et non supervisé, ainsi que des outils pour la préparation et la visualisation des données. Voici une liste non exhaustive des algorithmes disponibles dans scikit-learn :

- Régression linéaire
- Régression logistique
- Arbre de décision
- Random Forest
- Gradient Boosting
- Réseau de neurones artificiels
- SVM (Support Vector Machine)
- kNN (k-Nearest Neighbors)
- k-means clustering
- PCA (Principal Component Analysis)
- NMF (Non-negative Matrix Factorization)
- T-SNE (t-Distributed Stochastic Neighbor Embedding)
- Naive Bayes

Il est important de noter que cette liste n'est pas exhaustive et que de nouveaux algorithmes sont régulièrement ajoutés à la bibliothèque.



